Note: This notebook records some code for image processing, but may not be used in the final training

## Image Augmentation

In [ ]:
import os
import cv2
import random

def augment_images(folder_path):
    # Get list of subfolders
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    for subfolder in subfolders:
        # Get list of images in the subfolder
        images = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.endswith(('.jpg', '.jpeg', '.png'))]

        # Loop through each image
        for image_path in images:
            # Read image
            img = cv2.imread(image_path)

            # Perform augmentation: rotation to a random angle
            angle = random.randint(1, 360)  # Generate a random angle between 1 and 360
            rotation_matrix = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), angle, 1)
            rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))

            # Construct output file name
            output_path = os.path.splitext(image_path)[0] + '_rotated.jpg'

            # Write augmented image to disk
            cv2.imwrite(output_path, rotated_img)

            print(f"Augmented image saved: {output_path}")

# Path to the folder with images
folder_path = "./Datasets/dataset_self/merged_resize_3"
augment_images(folder_path)


## Mean value subtraction

In [ ]:
import os
import numpy as np
from PIL import Image
from torchvision import transforms

# Path to your dataset
dataset_path = "./Datasets/dataset_self/merged_resize_2/"

# Initialize variables to accumulate sum and sum of squares
mean_sum = np.zeros(3)
std_sum = np.zeros(3)
num_images = 0

# Iterate through the dataset
for class_name in os.listdir(dataset_path):
    if class_name.endswith(".DS_Store"):
        continue
    class_path = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(class_path, filename)
            img = Image.open(image_path)
            img = img.resize((227, 227))  # Adjust the size for input
            img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
            mean_sum += img_array.mean(axis=(0, 1))
            std_sum += img_array.std(axis=(0, 1))
            num_images += 1

# Calculate the mean and standard deviation
mean_values = mean_sum / num_images
std_values = std_sum / num_images

print("Mean values:", mean_values)
print("Standard deviation values:", std_values)